# Applying Euclidean distance on chord progressions normalized and Contious Spotify features

In [22]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [23]:
from scipy import stats

def applyLogTransformation(df, features, suffix):
    newFeatures = []
    for feature in features:
#         df[feature + '_log'] = np.sqrt(df[feature])
    
        df[feature + suffix] = stats.boxcox(df[feature])[0]
        newFeatures.append(feature + suffix)
        print('skewnees before:', df[feature].skew(), ' after: ', df[feature + suffix].skew())
        
    createKDeplot(df,newFeatures)

In [38]:
continousFeatures = pd.read_csv('./data/continousFeatures.csv')
continousFeatures = continousFeatures.sort_values('id')
continousFeatures.id = continousFeatures.id.astype('int64')
continousFeatures = continousFeatures.set_index('id')

In [39]:
continousFeatures

,danceability,energy,speechiness,acousticness,liveness,valence,tempo
id,,,,,,,
23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337
105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344
118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622
120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097
292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932
...,...,...,...,...,...,...,...
5000017,0.453,0.295,0.0291,0.518000,0.2930,0.140,146.022
5000018,0.525,0.292,0.0260,0.910000,0.0795,0.133,75.009
5000019,0.549,0.283,0.0284,0.661000,0.3570,0.163,76.992


In [34]:
transitionMatrices = pd.read_csv('./data/transitionMatrices.csv')
transitionMatrices = transitionMatrices.iloc[:, 1:]
transitionMatrices = transitionMatrices.sort_values('id')
transitionMatrices.id = transitionMatrices.id.astype('int64')
transitionMatrices = transitionMatrices.set_index('id')

In [35]:
transitionMatrices

,0,1,2,4,5,6,7,8,9,11,...,1276,1278,1281,1286,1287,1288,1290,1292,1294,1295
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
105,0.185185,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
118,0.000000,0.0,0.0,0.0,0.0,0.044944,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
120,0.029412,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
292,0.013889,0.0,0.0,0.0,0.0,0.000000,0.013889,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000025,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5000026,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.010638,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5000027,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [71]:
df = transitionMatrices.merge(continousFeatures, left_index=True, right_index=True)
df

,0,1,2,4,5,6,7,8,9,11,...,1292,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337
105,0.185185,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344
118,0.000000,0.0,0.0,0.0,0.0,0.044944,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622
120,0.029412,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097
292,0.013889,0.0,0.0,0.0,0.0,0.000000,0.013889,0.0,0.0,0.0,...,0.0,0.0,0.0,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000017,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.453,0.295,0.0291,0.518000,0.2930,0.140,146.022
5000018,0.032258,0.0,0.0,0.0,0.0,0.016129,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.525,0.292,0.0260,0.910000,0.0795,0.133,75.009
5000019,0.084746,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.549,0.283,0.0284,0.661000,0.3570,0.163,76.992


In [72]:
df['id'] = df.index

In [73]:
df[df.id == 1686718]

,0,1,2,4,5,6,7,8,9,11,...,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo,id
id,,,,,,,,,,,,,,,,,,,,,
1686718,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.334,0.252,0.0287,0.722,0.0907,0.214,153.879,1686718


In [74]:
df.reset_index(drop=True, inplace=True)

,0,1,2,4,5,6,7,8,9,11,...,1292,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo
count,3200.000000,3200.000000,3.200000e+03,3.200000e+03,3200.000000,3200.000000,3200.000000,3.200000e+03,3.200000e+03,3200.000000,...,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000
mean,0.012486,0.000001,1.002968e-07,1.454720e-08,0.000006,0.000240,0.000018,4.756871e-08,1.093532e-08,0.002745,...,1.000157,1.000043,1.000025,1.529714,1.546700,1.038659,1.353953,1.181865,1.435224,120.877524
std,0.016210,0.000016,2.314445e-06,5.817970e-07,0.000058,0.000863,0.000124,1.344815e-06,3.570344e-07,0.005491,...,0.003328,0.002411,0.000648,0.142138,0.224248,0.012868,0.322385,0.159394,0.229138,28.704911
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.091600,1.001880,1.022400,1.000006,1.019700,1.034500,44.260000
25%,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.431000,1.373000,1.029800,1.047650,1.097075,1.250000,97.596500
50%,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.533000,1.549000,1.034500,1.256000,1.118000,1.397000,120.961500
75%,0.025487,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.631000,1.728250,1.043300,1.644250,1.198000,1.595000,140.607000
max,0.044705,0.000181,5.349165e-05,2.327552e-05,0.000600,0.003398,0.000893,3.805497e-05,1.166434e-05,0.015491,...,1.150000,1.136364,1.020833,1.946000,1.988000,1.088700,1.996000,1.991000,1.979000,211.813000


In [75]:
ids = df.id
df = df.iloc[:, 0:-1]

## Removing Skewness with boxcox

In [76]:
df = df.loc[:,(df!=df.iloc[0]).any()]
df = df + 1
cols = df.columns.tolist()

In [77]:
for colName in cols:
    df[colName] = stats.boxcox(df[colName])[0]

In [78]:
df

,0,1,2,4,5,6,7,8,9,11,...,1292,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.532649,0.829228,0.016011,0.001398,0.086839,0.388442,15.528785
1,0.043706,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.634870,0.815061,0.014911,0.000458,0.033886,0.423975,15.529165
2,0.000000,0.0,0.0,0.0,0.0,0.003398,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.672540,0.828046,0.013855,0.015928,0.058576,0.412220,13.077161
3,0.021330,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.335619,0.889758,0.015579,0.001039,0.077504,0.393662,14.671014
4,0.011868,0.0,0.0,0.0,0.0,0.000000,0.000893,0.0,0.0,0.0,...,0.0,0.0,0.0,0.534950,0.564002,0.014955,0.161696,0.079696,0.226398,16.700026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.485728,0.307113,0.014153,0.334321,0.107112,0.124532,16.199573
3196,0.022730,0.0,0.0,0.0,0.0,0.003368,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.568426,0.303875,0.013508,0.462177,0.056822,0.118960,11.817227
3197,0.037459,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.596299,0.294174,0.014017,0.388330,0.111960,0.142419,11.967216
3198,0.000000,0.0,0.0,0.0,0.0,0.003398,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.531500,0.280198,0.013896,0.193973,0.074841,0.152252,11.662958


## Combining Centering and Scaling

In [79]:
cols = df.columns
scaler = StandardScaler()
scaler.fit(df)
data_normalized = scaler.transform(df)
print('mean: ', data_normalized.mean(axis=0).round(2))
print('std: ', data_normalized.std(axis=0).round(2))

df = pd.DataFrame(data_normalized, columns=cols)

mean:  [-0.  0. -0. -0.  0. -0.  0.  0. -0.  0. -0.  0.  0.  0. -0.  0. -0. -0.
  0.  0.  0. -0.  0.  0. -0.  0. -0.  0. -0.  0.  0. -0. -0. -0. -0. -0.
  0. -0.  0. -0.  0. -0.  0. -0.  0. -0.  0.  0. -0.  0. -0.  0. -0. -0.
  0. -0.  0. -0. -0.  0. -0. -0. -0.  0.  0.  0. -0.  0. -0.  0.  0. -0.
  0. -0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.  0.  0.  0.  0. -0. -0.
 -0. -0.  0.  0. -0. -0.  0.  0. -0.  0.  0. -0. -0. -0. -0.  0.  0. -0.
 -0.  0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.
  0. -0.  0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  0.  0.
  0.  0.  0. -0.  0.  0. -0. -0.  0. -0. -0. -0.  0.  0. -0. -0.  0.  0.
 -0. -0.  0. -0.  0.  0. -0. -0. -0. -0.  0.  0. -0. -0. -0.  0.  0. -0.
 -0.  0. -0. -0.  0.  0.  0. -0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0.
  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0.  0. -0. -0. -0.
  0. -0. -0.  0. -0. -0. -0.  0.  0. -0.  0. -0. -0. -0. -0.  0. -0. -0.
  0.  0. -0. -0. -0. -0. -0.  0.  0. -0. -0.

In [80]:
df

,0,1,2,4,5,6,7,8,9,11,...,1292,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo
0,-0.770369,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-0.267096,0.933855,0.755548,-1.283028,0.372405,0.763209,0.522885
1,1.926224,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,0.354638,0.878105,-0.227137,-1.288644,-2.173397,1.063184,0.523113
2,-0.770369,-0.090507,-0.043342,-0.025008,-0.098862,3.662358,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,0.583761,0.929205,-1.169929,-1.196220,-0.986407,0.963947,-0.945757
3,0.545668,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-1.465492,1.172049,0.369475,-1.285173,-0.076390,0.807277,0.009039
4,-0.038105,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,7.056658,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-0.253104,-0.109854,-0.187810,-0.325365,0.029006,-0.604754,1.224516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,-0.770369,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-0.552487,-1.120753,-0.903690,0.705940,1.347020,-1.464708,0.924720
3196,0.632050,-0.090507,-0.043342,-0.025008,-0.098862,3.626814,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-0.049491,-1.133495,-1.479803,1.469786,-1.070722,-1.511748,-1.700519
3197,1.540808,-0.090507,-0.043342,-0.025008,-0.098862,-0.278445,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,0.120039,-1.171669,-1.025145,1.028604,1.580113,-1.313707,-1.610668
3198,-0.770369,-0.090507,-0.043342,-0.025008,-0.098862,3.662368,-0.141714,-0.035377,-0.030633,-0.499944,...,-0.075212,-0.01768,-0.039559,-0.274090,-1.226669,-1.133120,-0.132531,-0.204428,-1.230691,-1.792933


## Applying eucledian distance

In [82]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [83]:
originalSongId = 180849 # The Scientist Original
originalSongIndex = ids[ids == originalSongId].index[0]
originalSongIndex

originalSongVector = df.iloc[originalSongIndex, :]

***Dropping the cover song***

In [84]:
coverSongId = 1686718 # THe Scientist by Boyce Avenue
coverIndex = ids[ids == coverSongId].index[0]
coverSongVector = df.iloc[coverIndex, :]

In [85]:
ranking = []
for index in range(0, len(df)):
    ranking.append([ids[index], euclidean_distance(df.iloc[index, :], originalSongVector)])
ranking     

[[23, 38.49315227763584],
 [105, 28.169470769692893],
 [118, 33.73969567603339],
 [120, 31.23282183569679],
 [292, 51.02274783394757],
 [407, 30.0365320387683],
 [418, 34.970743694456104],
 [456, 28.088649216692446],
 [467, 50.15921277897536],
 [469, 37.84599855441472],
 [635, 27.26427320848626],
 [652, 32.880143030647005],
 [816, 28.961798180121537],
 [830, 59.626293996407775],
 [836, 29.01836462347899],
 [1334, 28.327847514760045],
 [1345, 27.448492183951082],
 [1395, 27.705545047194313],
 [1433, 28.407889552990557],
 [2147, 31.19462991125506],
 [2148, 39.62055294327012],
 [2220, 28.768986498304564],
 [2535, 27.279295899583918],
 [3118, 30.18956648573351],
 [3165, 27.051857937552892],
 [3337, 27.44651403821615],
 [3991, 27.10474072533623],
 [4019, 27.638425741425216],
 [4041, 27.550592756304017],
 [4084, 28.131559133140996],
 [4169, 32.89111990328085],
 [5643, 27.688728544895522],
 [5909, 28.514592419262215],
 [6090, 28.595547471500844],
 [6097, 29.344842017737765],
 [6109, 32.852477

In [86]:
ranksDF = pd.DataFrame(ranking)

In [87]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [88]:
ranksDF

,id,rank
0,23,38.493152
1,105,28.169471
2,118,33.739696
3,120,31.232822
4,292,51.022748
...,...,...
3195,5000017,26.696636
3196,5000018,28.809071
3197,5000019,26.632725
3198,5000020,63.743647


In [89]:
ranksDF = ranksDF.sort_values(by="rank")

In [90]:
ranksDF.reset_index(drop=True, inplace=True)

In [91]:
ranksDF

,id,rank
0,180849,0.000000
1,1119299,21.774160
2,610562,24.236820
3,748267,25.928225
4,1127345,25.991993
...,...,...
3195,686587,161.105292
3196,1716013,170.498292
3197,79223,180.639978
3198,1201859,184.943784


In [93]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
193,1686718,26.995686


## Rank for original Song 450

In [94]:
ranksDF[ranksDF['id'] > 5000000]

,id,rank
16,5000019,26.632725
26,5000017,26.696636
177,5000013,26.984489
411,5000012,27.181602
1193,5000011,27.718482
1887,5000018,28.809071
2428,5000001,32.346915
3045,5000020,63.743647
3069,5000008,67.380534
3114,5000007,76.275717
